# Serialbox Tutorial

This notebook will walk through an example to have developers get familiar with the basic Serialbox features to save data from Fortran and load the data into Python.

The developer will write their Fortran code below the `%%writefile serialBox_tutorial.F90` line in the following cell.  After the Fortran code is completed, running this cell will write a Fortran file `serialBox_tutorial.F90` that can later be compiled.

The developer will do the following in the Fortran code with Serialbox.

* Initialize Serialbox, have the serialized data written to a directory called `./data`, and set the Serialbox file prefix to `example`.
* Create a Serialbox savepoint called `input_data` that contains all the serialized data
* Create an integer scalar that has a value equal to 7 and write it into a Serialbox variable called `int0`
* Create a real scalar that has a value equal to 8.9 and write it into a Serialbox variable called `real0`
* Create a 2D double precision array of size (10,11) where the value at the `(i,j)` index is computed as `(j-1) + i + 0.1` when looping through `i` and `j`.  Save the array using a Serialbox variable called `dp_arr0`

For a helpful reference, see slide 8.
Note the API for serialbox:

* !$ser init directory='Location for data files' prefix='Prefix for data files' - Call to initialize Serialbox

* !$ser savepoint 'SavepointName' - Create a new ‘savepoint’ to collect data

* !$ser data var_othername=var_fortranname - Save fields with simple syntax

* !$ser verbatim if x > 0.0 then … - Execute additional Fortran code when utilizing Serialbox

* !$ser <on/off> - Turn serializing on or off for different segments of code

In [48]:
%%writefile serialBox_tutorial.F90

program serialBox_tutorial

    implicit none

    integer          :: int0, ii, jj
    real             :: real0

    real, dimension(:,:), allocatable :: dp_arr0

    ! Initialize Serialbox
    !$ser init directory='./data' prefix='example' unique_id=.true.
    !$ser mode write
    !$ser on
    
    ! Set up the data as indicated in the above cell
    int0  = 7
    real0 = 8.9

    allocate(dp_arr0(10,11))

    do jj = 1, 11
        do ii = 1, 10
            dp_arr0(ii,jj) = (jj-1) + ii + 0.1
            !write(*,*) 'dp_arr0(', ii, ',', jj, ') = ', dp_arr0(ii,jj)
        enddo
    enddo
    ! Write out the data as indicated in the above cell using Serialbox
    !$ser savepoint 'input_data'
    !$ser data int0=int0 real0=real0
    !$ser data dp_arr0=dp_arr0

    !!$ser cleanup
   
end program

Overwriting serialBox_tutorial.F90


Once the Fortran file is written, the developer can run the cell below which creates a Bash environment to execute the `pp_ser.py ` Python script.  This will create a Fortran file `s_serialBox_tutorial.F90` with the appropriate SerialBox library calls in the Fortran code.  After that, the code is compiled using `gfortran` and executed.

In [49]:
%%bash

[ -f tutorial_run ] && rm tutorial_run
[ -f s_serialBox_tutorial.F90 ] && rm s_serialBox_tutorial.F90

python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_serialBox_tutorial.F90 serialBox_tutorial.F90

gfortran -O3 -cpp -DSERIALIZE \
    -o tutorial_run s_serialBox_tutorial.F90 \
    -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
    -lpthread -lstdc++ -lstdc++fs
./tutorial_run

Processing file serialBox_tutorial.F90
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
 >>> WARNING: SERIALIZATION IS ON <<<
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<


Once the binary executes and writes out the data, the developer can run the following cell containing a Python script to verify whether the data serialized properly or not.  The script assumes that the developer has set up the data exactly as specified in bulleted list above.  If the only message printed is `Finished running comparison tests!`, all the tests have passed!

In [50]:
#!/usr/bin/env python3

import numpy as np
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser

serializer = ser.Serializer(ser.OpenModeKind.Read,'./data', 'example')

sp = serializer.get_savepoint('input_data')

int0  = serializer.read('int0',  sp[0])[0]
real0 = serializer.read('real0', sp[0])[0]

dp_arr0   = serializer.read('dp_arr0',   sp[0])

int0_ref = 7
real0_ref = np.float32(8.9)

dp_arr0_ref = np.zeros((10,11), dtype=np.float32)

for j in range(11):
    for i in range(10):
        dp_arr0_ref[i,j] = j + i+1 + 0.1
try:
    assert int0_ref == int0, "int0 does not match!"
    assert real0_ref == real0, "real0 does not match!"
    assert np.array_equal(dp_arr0_ref, dp_arr0), "dp_arr0 does not match!"
except AssertionError as msg:
    print(msg)
print("Finished running comparison tests!")

Finished running comparison tests!


## Optional Exercise 2
